# Data Wrangling with Python
Fall 2021

In this workshop, we'll dive deep into some techniques for cleaning and re-shaping data in Python using the [pandas](https://pandas.pydata.org/docs/) library. 

Here's what you can expect to practice:
 - working with CSV data from various sources
 - reshaping data for analysis
 - joining datasets on common elements
 - handling text and time series data
 - dealing with nulls and duplicates

## Research question

Calculate change in US home prices over a five-year period, using the Zillow Home Value Index, and compare with change in median household income. Identify regions where the two measures diverge.

## Data sources

 - [Zillow Home Value Index](https://www.zillow.com/research/data/): "A smoothed, seasonally adjusted measure of the typical home value and market changes across a given region and housing type. It reflects the typical value for homes in the 35th to 65th percentile range."
 - [US Census, American Community Survey](https://www.census.gov/programs-surveys/acs) data for median household income over the previous 12 months.

## Setting up

### Library imports

Let's import any libraries we'll need. 

We'll do most of our work in `pandas`, which should be available automatically in a Google Colab environment. 

In [1]:
import pandas as pd

We'll also use a library that simplifies the process of retrieving Census data. We may need to install it first.

In [2]:
from census import Census

### Links & API registration

At this time, you should also [register for an API key](https://api.census.gov/data/key_signup.html) so as to be able to retrieve datasets from census.gov. Once you have completed the form, you should receive your API key at the email address you provided. 

The following link will allows us to download Zillow Home Value Index (ZVHI) data. The data covers the time period between January 2000 and August 2021. Values are aggregated at the level of the zip code.

In [3]:
zhvi_all_homes = 'https://files.zillowstatic.com/research/public_csvs/zhvi/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1631541070'

## Loading & exploring data

### Loading data from CSV

The pandas `read_csv` method can load data from a URL as well as a file, provided the data is in the proper format. 

In [4]:
zhvi_df = pd.read_csv(zhvi_all_homes)

### Reshaping data

If we look at the DataFrame's columns, we can see that each month is represented by a discrete column heading, leading to a very wide table. This is done to compress the data to save storage space. (The names of the months appear only once, in the column headings.) 

In [5]:
zhvi_df.columns

Index(['RegionID', 'SizeRank', 'RegionName', 'RegionType', 'StateName',
       'State', 'City', 'Metro', 'CountyName', '2000-01-31',
       ...
       '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
       '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31', '2021-08-31'],
      dtype='object', length=269)

The table has 30K+ rows, one row per zip code.

In [6]:
len(zhvi_df)

30523

For analysis, it's often easier to work with data in so-called "long" format. Meaning that each row corresponds to a single observation. 

An observation is a specific measure of one or more variables. In this dataset, we have basically two variables: time and location (zip code). So how can we re-shape this data so that each row contains a single value for ZHVI? 

pandas has a handy method called `melt`, which is useful when you have several columns that all contain the same **kind** of measure. Here, each of the month columns contains the same measure, i.e., the ZHVI values for that month. So we can "melt" this table so that all the ZHVI values are in one column, and all the months in another. 

And what about the geographical columns? We will retain those, making them what pandas calls "ID variables" -- the idea being that the combination of those columns creates a set of unique identifiers. In this case, that's the geographical location identified by zip code. 

In the following method call, I'm referring to the geographical columns by position, which I can do since they precede all of the month columns, by slicing the DataFrame's `columns` object.

In [7]:
z_df = zhvi_df.melt(id_vars=zhvi_df.columns[0:9], var_name='month', value_name='value')

Now our dataset is much bigger, though it has exactly the same data! But the new shape will make it easier to group our data and filter it in different way. 

In [8]:
len(z_df)

7935980

### Correcting data types

It's useful to make sure the datatypes in your DataFrame make sense for what they represent. In this case, our zip codes should be strings, our values floats, and our months datetime objects.

The `dtypes` property displays the type of each column. In pandas, a type of `object` is either for a column of strings or of mixed data types.

In [9]:
z_df.dtypes

RegionID        int64
SizeRank        int64
RegionName      int64
RegionType     object
StateName      object
State          object
City           object
Metro          object
CountyName     object
month          object
value         float64
dtype: object

#### Padding strings 

Let's convert the zip code fields to strings. One thing you might notice is that by treating the zip code (`RegionName`) as an integer, pandas has truncated zip codes that begin with zero. That could pose problems later, if we want to match this data against other data using the zip code. 

We can fix that by converting the `RegionName` column to a string and using the Python string method `zfill`, which adds zeroes to the beginning of a string to make it the required length. The argument to `zfill` is the **total number** of characters in the string. If the string to which you apple `zfill` contains fewer characters than the number you provide, the string will be padded with zeroes to fill it out to the required length. For example, `'7'.zfill(3)` returns `'007'`.

To use `zfill` on the `RegionName` column of our DataFrame, we first have to convert the data to strings, using the `astype` method. Then we can `apply` the `zfill` method to the data in that column. The pandas `apply` method, which takes as its argument another function, executes that function once for each value in the column. 

In [10]:
z_df['RegionName'] = z_df['RegionName'].astype(str).apply(lambda x: x.zfill(5))

#### Converting dates and times

Now let's convert the `month` column to a `datetime` type. This makes it much easier to aggregate on time series. In this case, pandas can interpret the string correctly without our supplying a pattern. In other cases, it may be necessary to provide a second argument to `pd.to_datetime`, indicating the pattern of the string.

In [12]:
z_df['month'] = pd.to_datetime(z_df['month'])

Now we can filter by parts of the date, for instance, by year. To do this, we use the special `dt` attribute, which has attributes corresponding to day, month, and year.

In [13]:
z_df.loc[z_df.month.dt.year == 2021]

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,month,value
7691796,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,2021-01-31,1121064.0
7691797,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,2021-01-31,503891.0
7691798,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,2021-01-31,1466975.0
7691799,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,2021-01-31,358995.0
7691800,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,2021-01-31,643988.0
...,...,...,...,...,...,...,...,...,...,...,...
7935975,66169,34430,20052,Zip,DC,DC,Washington,Washington-Arlington-Alexandria,District of Columbia,2021-08-31,1443140.0
7935976,89666,34430,72630,Zip,AR,AR,Diamond City,Harrison,Boone County,2021-08-31,118818.0
7935977,62532,34430,12345,Zip,OR,OR,Central Point,Medford,Jackson County,2021-08-31,169411.0
7935978,87060,34430,66045,Zip,KS,KS,Lawrence,Lawrence,Douglas County,2021-08-31,256756.0


#### Customizing the numerical format

When dealing with dollar values in the millions, it is useful to limit the number of decimal places to 2 and to add commas separating the thousands, etc. We can set this option globally as follows.

Note that it does not affect the underlying representation of the data (which is still floating-point decimal), just the way it displays on screen.

In [14]:
pd.set_option('display.float_format','{:,.2f}'.format)

In [15]:
z_df

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,month,value
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,2000-01-31,"329,121.00"
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,2000-01-31,"311,929.00"
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,2000-01-31,"500,887.00"
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,2000-01-31,"225,325.00"
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,2000-01-31,"405,926.00"
...,...,...,...,...,...,...,...,...,...,...,...
7935975,66169,34430,20052,Zip,DC,DC,Washington,Washington-Arlington-Alexandria,District of Columbia,2021-08-31,"1,443,140.00"
7935976,89666,34430,72630,Zip,AR,AR,Diamond City,Harrison,Boone County,2021-08-31,"118,818.00"
7935977,62532,34430,12345,Zip,OR,OR,Central Point,Medford,Jackson County,2021-08-31,"169,411.00"
7935978,87060,34430,66045,Zip,KS,KS,Lawrence,Lawrence,Douglas County,2021-08-31,"256,756.00"


### Exploring the data

#### Null values

One thing we may want to know about dataset is how many rows have null values. There are other ways to accomplish this, but a straightforward way to get a total by column is to call `sum` on the result of the `DataFrame.isna` method. `isna` returns a new DataFrame where each cell is either `True` or `False`, depending on whether it's null or not. And the `sum` method simply "adds up" these Boolean values, treating `True` as 1 and `False` as 0. The result shows us how many null values are in each column.

In [21]:
z_df.isna().sum()

RegionID            0
SizeRank            0
RegionName          0
RegionType          0
StateName           0
State               0
City                0
Metro         2248468
CountyName          0
month               0
value         1817353
dtype: int64

We can see that the only columns with nulls are the `Metro` column and the `value` column. The `Metro` column refers to the zip code's metro area; rural zip codes would not have a metro area, so that makes sense. The nulls in the `value` column might be more problematic, depending on our analysis; these are instances where we don't have a valid observation. (Maybe no data were available.)

Here we also have an example of the principle that an aggregate operation on a DataFrame (`sum`) returns a pandas Series.

If we want to examine the null values, we can use the `isna` method to find them.

In [18]:
z_df.loc[z_df.value.isna()]

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,month,value
22,92271,23,78130,Zip,TX,TX,New Braunfels,San Antonio-New Braunfels,Comal County,2000-01-31,nan
24,61643,25,10029,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,2000-01-31,nan
26,92036,27,77573,Zip,TX,TX,League City,Houston-The Woodlands-Sugar Land,Galveston County,2000-01-31,nan
35,92515,36,78572,Zip,TX,TX,Mission,McAllen-Edinburg-Mission,Hidalgo County,2000-01-31,nan
53,86026,54,63376,Zip,MO,MO,Saint Peters,St. Louis,Saint Charles County,2000-01-31,nan
...,...,...,...,...,...,...,...,...,...,...,...
7927728,81919,23362,54836,Zip,WI,WI,Foxboro,Duluth,Douglas County,2021-08-31,nan
7928350,61885,24090,10597,Zip,NY,NY,Town Of Lewisboro,New York-Newark-Jersey City,Westchester County,2021-08-31,nan
7928562,81907,24347,54820,Zip,WI,WI,Brule,Duluth,Douglas County,2021-08-31,nan
7928676,81935,24479,54854,Zip,WI,WI,Maple,Duluth,Douglas County,2021-08-31,nan


#### Checking for duplicates

Another problem frequently encountered is duplicated data. We can check for that using the `duplicated` method, which works like `isna` except that it returns `True` if a value is duplicated within a column. By default, it doesn't mark the first duplicate as a duplicate, but I usually find it helpful to see all the duplicates, so I pass the parameter `keep=False`, which means, "Count the first instance of a duplicated datum as a duplicate."

In [20]:
zhvi_df.loc[zhvi_df.RegionName.duplicated(keep=False)]

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2000-01-31,...,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31,2021-08-31


No zip codes in our original dataset are duplicated, so that's good! Note that I didn't call that method on our "melted" dataset, since converting the original table to a "long" version created many duplicates. 

In our "melted" dataset, the combination of zip code and month should be unique. We can check for this using the `subset` parameter.

In [21]:
z_df.loc[z_df.duplicated(subset=['RegionName', 'month'], keep=False)]

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,month,value


### Getting ACS data

To make things more interesting, we're going to combine our Zillow data with a dataset from the US Census. The American Community Survey provides five-year estimates of many important economic indicators, including median household income by zip code. 

We can use the Census API to fetch the tables we need. The Python [census](https://pypi.org/project/census/) package provides a convenient wrapper around the API's. 

If you submitted the form at the start of our workshop, you should have received an API key via the email address you provided. If you didn't, I'll provide a link to the datasets we're going to use. 

#### Retrieving Census data 

First we need to initialize the module with our API key.

In [23]:
apikey = '0009292cbc84afabcef7b434345abb0df20349c7'
census = Census(apikey)

To access Census tables by API, we need to know the specific variable names. You can find these in the [ACS documentation](https://api.census.gov/data/2019/acs/acs5/variables.html). Using the `census` module, we can specify the ACS 5-year tables and zip code as the geographical level. In addition, you can specify a year as a method parameter, if we want data other than that from the most recent survey. 

In order to have some timeseries data, we'll retrieve the data for 2014 and 2019. (It's recommended to use [non-overlapping datasets](https://www.census.gov/data/developers/data-sets/acs-5year.html) for the ACS 5-year surveys because of how the estimates are calculated.)

In [25]:
median_income = "B19013_001E"
census = Census(apikey)
acs2019 = census.acs5.zipcode(median_income, '*')
acs2014 = census.acs5.zipcode(median_income, '*', year=2014)

#### Creating a DataFrame

The Census API returns the table as a list of dictionaries. You can see that the value for our variable -- median household income -- is provided for each row, as well as a code for the state and the zip code (both strings). 

Our first step is to convert these two lists to DataFrames so that we can efficiently merge them with our Zillow data.

In [ ]:
acs2019

pandas has a handy method call `DataFrame.from_records`, which creates a DataFrame from exactly this structure. (The elements of the list are the rows, the keys of the dictionaries are the columns.)

We can also rename our columns for clarity and concision.

In [27]:
acs_df = pd.DataFrame.from_records(acs2019)
acs_df = acs_df.rename(columns={'B19013_001E': '2019_median_hhi',
                               'zip code tabulation area': 'zip_code'})

In [28]:
len(acs_df)

33120

#### Concatenating DataFrames

But we have two separate datasets, one from 2019 and one from 2014. Ideally, we want a single table containing all of our ACS data. We'll accomplish that by using pandas' `concat` method, which takes a list of DataFrames and stacks them one on top of the other.

We'll add a column to record the year. And we'll put this code into a Python function, which is good practice for encapsulating our code for reproducibility.

In [29]:
def create_acs_df(datasets, years):
    '''
    Accepts a list of pandas DataFrames to concat and a list of years, which will be added as a column to the resulting DataFrame
    '''
    # Step 1: Create an empty DataFrame -- for the first dataset, we need something to concat it with
    df_all = pd.DataFrame()
    # Step 2: Create a for loop: we can use the zip method to loop over the datasets and years at the same time
    for dataset, year in zip(datasets, years):
        # Create a DataFrame from the current dataset
        df = pd.DataFrame.from_records(dataset)
        # Create a year column and populate it with the corresponding year
        df['year'] = year
        # Rename the columns
        df = df.rename(columns={'B19013_001E': 'median_hhi',
                               'zip code tabulation area': 'zip_code'})
        # Concat with the previous DataFrame
        df_all = pd.concat([df_all, df])
    # Don't forget to return something!
    return df_all

In [30]:
acs_df = create_acs_df([acs2019, acs2014], [2019, 2014])

In [31]:
acs_df

,median_hhi,state,zip_code,year
0,"13,092.00",72,00601,2019
1,"16,358.00",72,00602,2019
2,"16,603.00",72,00603,2019
3,"12,832.00",72,00606,2019
4,"19,309.00",72,00610,2019
...,...,...,...,...
33115,"-666,666,666.00",02,99923,2014
33116,"39,167.00",02,99925,2014
33117,"51,071.00",02,99926,2014
33118,"17,946.00",02,99927,2014


#### Dealing with "bad" data

It looks like we have some outliers here, which may be an artifact of how the data has been reported/coded. We can use the `describe` method to try to spot these. (Ignore the year column; we're only interested in outliers in the `median_hhi` column.

In [117]:
acs_df.describe()

,median_hhi,year
count,66205.00,66240.00
mean,-33237571.79,2016.50
std,145221476.17,2.50
min,-666666666.00,2014.00
25%,38750.00,2014.00
50%,50056.00,2016.50
75%,63826.00,2019.00
max,250001.00,2019.00


It doesn't really make sense to have negative median income; and indeed, by looking at the unique values in the column that fall **below** zero, we can see that there's only one value, which may mean any number of things, but for the purposes of this example, we can just discard those rows.

In [33]:
acs_df.loc[acs_df.median_hhi < 0].median_hhi.unique()

array([-6.66666666e+08])

We're using `DataFrame.loc` to limit to the rows with a non-negative value for `median_hhi`. We'll make a copy of the filtered DataFrame, which helps avoid issues as we manipulate this data later.

In [34]:
acs_df = acs_df.loc[acs_df.median_hhi >= 0].copy()

In [35]:
acs_df.describe()

,median_hhi,year
count,"62,899.00","62,899.00"
mean,"55,749.01","2,016.46"
std,"24,176.37",2.50
min,"2,499.00","2,014.00"
25%,"40,623.00","2,014.00"
50%,"51,125.00","2,014.00"
75%,"64,821.00","2,019.00"
max,"250,001.00","2,019.00"


### Building our joint dataset

In order to compare Zillow home values and median household income by geographical area, it will be useful to have a single table that contains both variables. We'll create this table in a few steps.

#### Creating aligned data

Most important is to make sure that our two variables are aligned, meaning that they represent observations at the same scale. 

How well are our datasets aligned?

| Dataset | Measure | Geographic Dimension | Temporal Dimension |
| --- | --- | --- | --- |
| Zillow | Estimated median home value (dollars) | Zipcode | Month |
| ACS | Median household income (dollars) | Zipcode | Year |

Zillow and ACS are measuring different things, but they're both using the median as their statistic, and they're both measuring value in dollars. Geographically, both datasets are aligned at the level of zipcode. 

However, the Zillow data are presented on a monthly basis, while the ACS data are available only at the annual level. Moreover, because of the 5-year overlap, we have only two "years" of ACS data to work with. We'll need to adjust our Zillow dataset to bring it into alignment with our ACS data.


(Technically, the ACS data represent five-year aggregates, which are not the same as annual measures. In what follows we are going to take a statistically naive approach to comparing and summarizing these data. Tis approach is not intended to be empirically rigorous; rather, it is meant to illustrate operations with pandas. For more on the appropriate methodology for handling ACS median values, [this guide](http://www.dof.ca.gov/Forecasting/Demographics/Census_Data_Center_Network/documents/How_to_Recalculate_a_Median.pdf) is a good starting point.)

#### Filtering a dataset and checking for missing data

First we limit the Zillow data to the years for which we have ACS data: 2014 and 2019. We can use the `dt` datetime attribute on the `month` column to select particular years with `DataFrame.loc`.

In [62]:
z_df = z_df.loc[(z_df.month.dt.year == 2019) | (z_df.month.dt.year == 2014)].copy()

To see if there are zipcodes represented in each dataset not present in the other, we can use the `asin` method to compare the values in one DataFrame with the values in another. This code counts how many unique zip codes are in our Zillow dataset that are not present in the ACS dataset

In [63]:
len(z_df.loc[~z_df.RegionName.isin(acs_df.zip_code)].RegionName.unique())

907

We can do the same to count how many zip codes in the ACS dataset are missing from the Zillow data.

In [64]:
len(acs_df.loc[~acs_df.zip_code.isin(z_df.RegionName)])

4640

In this case, we'll simply omit these non-overlapping data points from our merged dataset. But in other cases, it might be important to account for them in some way.

#### Aggregating by time scale

We'll aggregate the Zillow data by year in order to compare with them the ACS data at the same time scale. 

We'll use the `DataFrame.groupby` method to group our dataset at the desired level. We're grouping by year, so we can use the `dt.year` attribute on the `month` column as the group key. We can group by multiple columns, so we might want to include other columns, too (except for the `value` column, which contains the data that we're trying to summarize).

We can provide them as a list to the `groupby` method, which use the unique combinations of the values in these columns as the keys for grouping.

In [65]:
z_grp = z_df.groupby([z_df.StateName, z_df.City, z_df.Metro, z_df.RegionName, z_df.month.dt.year])

Now let's take the `mean` of the `value` column from our grouped DataFrame. Such operations by default return a Series (in this case, with a so-called hierarchical index). But to turn that back to a DataFrame, we can use the `reset_index` method.

In [66]:
z_means = z_grp.value.mean()
z_means = z_means.reset_index()

In [67]:
z_means

,StateName,City,Metro,RegionName,month,value
0,AK,Anchorage,Anchorage,99501,2014,"265,894.50"
1,AK,Anchorage,Anchorage,99501,2019,"277,092.83"
2,AK,Anchorage,Anchorage,99502,2014,"295,827.08"
3,AK,Anchorage,Anchorage,99502,2019,"325,150.25"
4,AK,Anchorage,Anchorage,99503,2014,"241,454.50"
...,...,...,...,...,...,...
46811,WY,Weston,Gillette,82731,2019,"370,148.83"
46812,WY,Wilson,Jackson,83014,2014,"1,191,844.42"
46813,WY,Wilson,Jackson,83014,2019,"1,576,990.58"
46814,WY,Wright,Gillette,82732,2014,"215,183.17"


When grouping by multiple columns, it's important to be careful when including columns with null values. Recall from our analysis above that sometimes the `Metro` column is blank in this dataset. 

What happens to those rows in our final group?

In [68]:
z_means.loc[z_means.Metro.isnull()]

,StateName,City,Metro,RegionName,month,value


By grouping by a column with nulls, we've actually lost a fair amount of data! 

In [69]:
len(z_df.loc[~z_df.RegionName.isin(z_means.RegionName)].RegionName.unique())

7115

To fix this problem, we could omit the `Metro` column from our `groupby` statement. In more recent versions of pandas, we can also add a `dropna=False` parameter to our `groupby` statement, which will keep the null values in the group keys.

In [70]:
z_grp = z_df.groupby([z_df.StateName, z_df.City, z_df.Metro, z_df.RegionName, z_df.month.dt.year],
                    dropna=False)
z_means = z_grp.value.mean()
z_means = z_means.reset_index()
z_means.loc[z_means.Metro.isnull()]

,StateName,City,Metro,RegionName,month,value
0,AK,Anchor Point,NaN,99556,2014,"153,405.92"
1,AK,Anchor Point,NaN,99556,2019,"169,179.08"
32,AK,Clam Gulch,NaN,99568,2014,"164,201.92"
33,AK,Clam Gulch,NaN,99568,2019,"181,696.50"
34,AK,Cooper Landing,NaN,99572,2014,"292,707.75"
...,...,...,...,...,...,...
61037,WY,Wheatland,NaN,82201,2019,"210,679.25"
61040,WY,Worland,NaN,82401,2014,"147,281.58"
61041,WY,Worland,NaN,82401,2019,"164,424.08"
61044,WY,Yoder,NaN,82244,2014,"177,986.25"


In [71]:
z_means

,StateName,City,Metro,RegionName,month,value
0,AK,Anchor Point,NaN,99556,2014,"153,405.92"
1,AK,Anchor Point,NaN,99556,2019,"169,179.08"
2,AK,Anchorage,Anchorage,99501,2014,"265,894.50"
3,AK,Anchorage,Anchorage,99501,2019,"277,092.83"
4,AK,Anchorage,Anchorage,99502,2014,"295,827.08"
...,...,...,...,...,...,...
61041,WY,Worland,NaN,82401,2019,"164,424.08"
61042,WY,Wright,Gillette,82732,2014,"215,183.17"
61043,WY,Wright,Gillette,82732,2019,"233,214.58"
61044,WY,Yoder,NaN,82244,2014,"177,986.25"


#### Merging data

Now we're ready to merge our Zillow data with our ACS data. The `merge` method performs the equivalent of a SQL join on two DataFrames that share at least one common key. The keys are the columns with values that are the same in both datasets. The following chart shows the keys between our two datasets.

| Measure or Dimension | `z_means` | `acs` |
| --- | --- | --- | 
| State | `StateName` |  | 
| City | `City` |  |
| Metro | `Metro` |  |
| Zip code | `RegionName` | `zip_code`  |
| Year | `month` | `year` |
| State code |  | `state` |
| Median income | | `median_hhi` |
| Home value | `value` |  |

The keys don't have to have the same column **names**, but they must share at least some of the same **values** in those columns. In our case, the shared values are the zip codes and the years. (Note that both our Zillow and ACS datasets have columns for U.S. state, but the values in those columns do not overlap: the Zillow dataset uses the name of the state, while the ACS data uses a numerical identifier.)

Before merging our datasets, we'll rename the Zillow columns `month` and `value` to more accurately represent their contents. This isn't necessary for merging, but it will make the merged table more legible for us.

In [74]:
z_means = z_means.rename(columns={'month': 'year',
                                 'value': 'zhvi'})


Now we do the merge, which creates a new DataFrame. The arguments to the `merge` method are as follows:

- `acs_df`: the second DataFrame we want to merge with the first.
- `left_on`: this parameter takes a list of columns in the first DataFrame to use as keys.
- `right_on`: this parameter takes a list of columns in the second DataFrame to use as keys.

You can also specify a `how` parameter, which indicates the kind of join. The default is an **inner** join, meaning that the merged DataFrame will only contains rows where the keys are present in both of the source DataFrames. In this case, an inner join will drop those rows for zip codes that are missing from either the Zillow or the ACS data.

In [75]:
z_merged = z_means.merge(acs_df, left_on=['RegionName', 'year'], 
                         right_on=['zip_code', 'year'])

We don't need the `state` column from our ACS dataset. Nor do we need two columns of zip codes, so we can drop these from our merged dataset. The `axis` parameter is important in the `drop` method: `axis=1` means "drop columns" (as opposed to rows).

In [76]:
z_merged = z_merged.drop(['state', 'zip_code'], axis=1)

#### Calculating change over time

As an illustration of further uses for `groupby`, we'll calculate the percentage change between 2014 and 2019 for both the ZHVI and median household income for each zip code.

First, we'll sort the merged dataset by year.

In [78]:
z_merged = z_merged.sort_values(by='year')

Now we can create new columns to hold the percentage change for each of our measures (home value and income). To calculate the percentage change, we will use the built-in `pct_change` method, applying it to the result of a `groupby` operation (since we still want to observe the changes at the zip-code level).

In [80]:
z_merged.columns

Index(['StateName', 'City', 'Metro', 'RegionName', 'year', 'zhvi',
       'median_hhi'],
      dtype='object')

In [81]:
columns = ['StateName', 'City', 'Metro', 'RegionName']
z_merged['zhvi_pc'] = z_merged.groupby(columns, dropna=False).zhvi.pct_change()
z_merged['hhi_pc'] = z_merged.groupby(columns, dropna=False).median_hhi.pct_change()

Note that the `pc` columns have null values for all the rows where the `year` is 2014. That is expected: the first value in each sequence represents no change (since it's the first value). For each subsequent value, the percentage change is calculated with respect to the preceding value.

In [82]:
z_merged

,StateName,City,Metro,RegionName,year,zhvi,median_hhi,zhvi_pc,hhi_pc
0,AK,Anchor Point,NaN,99556,2014,"153,405.92","43,906.00",nan,nan
24222,MI,Hancock,Houghton,49958,2014,"74,156.25","36,731.00",nan,nan
46303,SC,Georgetown,Georgetown,29440,2014,"124,045.83","34,117.00",nan,nan
24220,MI,Hancock,Houghton,49952,2014,nan,"42,917.00",nan,nan
24218,MI,Hancock,Houghton,49930,2014,"104,142.17","36,880.00",nan,nan
...,...,...,...,...,...,...,...,...,...
25985,MN,Greenfield,Minneapolis-St. Paul-Bloomington,55357,2019,"427,492.25","104,821.00",0.23,0.19
25987,MN,Greenwald,St. Cloud,56335,2019,"129,383.67","57,708.00",0.23,0.18
25989,MN,Grey Eagle,NaN,56336,2019,"214,723.42","58,688.00",0.24,0.05
25993,MN,Grygla,Bemidji,56727,2019,"113,225.83","52,656.00",0.38,0.02


If we're just interested in the percentage change, we could drop the 2014 rows.

In [84]:
z_merged = z_merged.loc[z_merged.year == 2019].copy()

In [85]:
z_merged.describe()

,year,zhvi,median_hhi,zhvi_pc,hhi_pc
count,"28,708.00","28,397.00","28,708.00","26,765.00","28,643.00"
mean,"2,019.00","229,401.67","59,869.23",0.26,0.14
std,0.00,"235,203.14","25,221.57",0.17,0.26
min,"2,019.00","14,309.33","2,499.00",-0.49,-0.89
25%,"2,019.00","108,478.50","43,720.25",0.16,0.03
50%,"2,019.00","166,515.58","54,514.50",0.25,0.11
75%,"2,019.00","266,548.00","69,135.00",0.34,0.21
max,"2,019.00","6,300,275.67","250,001.00",2.68,9.32


And here's a demonstration of how we could use a custom function to calculate percentage change. There's no need to do so -- the built-in method will be more efficient. But sometimes you want to compute an aggregation that's not available as a built-in method.

For relatively simple calculations, we can use a `lambda` function, which is basically a one-line Python function. We define it with `lambda` instead of `def`. 

The `x` in the lambda expression is a function parameter. When used with `apply` on the `zhvi` column, `x` will represent a pandas Series containing the ZHVI values for each group created by our `groupby` expression. 

Since `x` is a Series, it has all the usual Series methods and properties, including `iloc`, which lets us take the first and second values for the purposes of calculating the percentage change.

In [87]:
z_means.groupby(['RegionName', 'StateName', 'City']).zhvi.apply(lambda x: (x.iloc[1] - x.iloc[0]) / x.iloc[0])

RegionName  StateName  City      
00501       NY         Holtsville   0.25
00602       AL         Auburn       0.29
00693       PA         Greensburg    nan
00705       UT         Aibonito     0.13
00734       TX         Ponce        0.98
                                    ... 
99714       AK         Salcha       0.11
99801       AK         Juneau       0.15
99824       AK         Juneau       0.11
99833       AK         Petersburg    nan
99901       AK         Ketchikan    0.13
Name: zhvi, Length: 30523, dtype: float64

We can now use our merged, aggregated dataset to investigate certain questions.

For example, where have home values risen *faster* than household income?

In [88]:
z_merged.loc[z_merged.zhvi_pc > z_merged.hhi_pc]

,StateName,City,Metro,RegionName,year,zhvi,median_hhi,zhvi_pc,hhi_pc
56985,WV,Benwood,Wheeling,26031,2019,"47,816.25","35,714.00",0.19,0.11
40722,OK,Atwood,NaN,74827,2019,"125,130.42","45,000.00",0.18,-0.19
57192,WV,Dunmore,NaN,24934,2019,"78,474.17","28,403.00",0.36,-0.15
50587,TX,Murphy,Dallas-Fort Worth-Arlington,75094,2019,"390,048.58","133,125.00",0.29,0.10
57184,WV,Dry Creek,Beckley,25062,2019,"59,717.83","28,250.00",0.20,-0.16
...,...,...,...,...,...,...,...,...,...
25983,MN,Greenbush,NaN,56726,2019,"76,219.67","59,167.00",0.35,0.11
25985,MN,Greenfield,Minneapolis-St. Paul-Bloomington,55357,2019,"427,492.25","104,821.00",0.23,0.19
25987,MN,Greenwald,St. Cloud,56335,2019,"129,383.67","57,708.00",0.23,0.18
25989,MN,Grey Eagle,NaN,56336,2019,"214,723.42","58,688.00",0.24,0.05


Where have they kept pace or lagged behind income?

In [89]:
z_merged.loc[z_merged.zhvi_pc <= z_merged.hhi_pc]

,StateName,City,Metro,RegionName,year,zhvi,median_hhi,zhvi_pc,hhi_pc
45477,PA,Tunkhannock Township,East Stroudsburg,18610,2019,"164,201.83","65,810.00",0.14,0.27
56916,WV,Alum Creek,Charleston,25169,2019,"46,390.50","44,115.00",0.04,0.92
40806,OK,Byars,Oklahoma City,74831,2019,"81,355.42","39,063.00",0.16,0.20
56909,WV,Alum Creek,Charleston,25003,2019,"102,873.75","37,224.00",-0.03,0.07
57226,WV,Evans,NaN,25241,2019,"134,118.08","85,451.00",0.16,0.43
...,...,...,...,...,...,...,...,...,...
26001,MN,Hamburg,Minneapolis-St. Paul-Bloomington,55339,2019,"217,925.25","88,333.00",0.26,0.34
26015,MN,Harmony,Rochester,55939,2019,"153,561.42","54,271.00",0.34,0.37
26023,MN,Hawley,Fargo,56549,2019,"228,107.25","79,191.00",0.21,0.25
25997,MN,Hackensack,Brainerd,56452,2019,"218,475.33","54,700.00",0.17,0.17


### Questions for practice

1. Which states have the greatest range in home values? Hint: select year, group by state, calculate max - min


2. Can you find any cities where the *average* percent change in median income is higher than the *average* percent change in home values?


3. Are the most expensive cities (to buy in) the wealthiest (by median income)?
-------------

1. Answer

   a. We need to pick a point in time: let's say 2019. 

In [90]:
z19 = z_means.loc[z_means.year == 2019]

1. b. Then we can group by state and use `apply` with a lambda function to find the difference between the largest and smallest values in each group. 

In [91]:
z_19_range = z19.groupby('StateName').zhvi.apply(lambda x: x.max() - x.min())

1. c. Finally, we can sort the values in descending order to find the biggest differences.

In [92]:
z_19_range.sort_values(ascending=False)

StateName
CA   6,271,268.00
NY   4,688,946.17
MA   4,152,406.17
WA   2,816,108.83
FL   2,631,860.42
CO   2,563,498.75
NJ   2,402,941.08
GA   2,389,238.42
NV   2,326,124.42
SC   1,949,929.25
MD   1,795,643.00
WY   1,533,227.92
AZ   1,529,388.50
NH   1,516,832.58
CT   1,471,269.50
IL   1,342,699.33
HI   1,221,715.00
ME   1,188,545.83
TX   1,185,032.42
KY   1,132,649.58
VA   1,125,548.75
UT   1,109,011.50
DC   1,060,699.08
DE   1,017,777.08
PA     981,529.50
NC     953,117.25
RI     877,937.42
MO     847,964.33
OR     744,450.00
MT     740,635.25
MN     739,840.58
IN     696,434.25
TN     692,581.67
ID     687,698.33
NM     685,007.83
MI     665,644.58
AL     558,244.25
KS     514,685.42
WI     477,869.25
OH     476,425.33
VT     472,841.67
LA     472,063.42
AK     467,253.50
NE     449,118.42
OK     402,821.25
AR     360,262.17
IA     358,763.17
ND     340,545.33
WV     309,750.58
SD     291,342.75
MS     255,029.92
Name: zhvi, dtype: float64

2. Answer

   a. We can group our merged dataset by city and state.

In [99]:
city_pc = z_merged.groupby(['City', 'StateName'])

2. b. To compute the mean of two columns at the same time, we can use the `agg` method available on a pandas `GroupBy` object. This method takes a dictionary mapping column names to the name of a built-in pandas method or a lambda function.

In [100]:
city_pc = city_pc.agg({'zhvi_pc': 'mean',
                             'hhi_pc': 'mean'})

2. c. Now we can use `.loc` to find those rows where our mean ZHVI percentage change is smaller.

In [101]:
city_pc.loc[city_pc.zhvi_pc < city_pc.hhi_pc]

zhvi_pc  hhi_pc
City              StateName                 
Abbot             ME            0.14    0.21
Aberdeen          MD            0.07    0.11
                  NC            0.17    0.35
Abington Township PA            0.12    0.18
Abita Springs     LA            0.07    0.24
...                              ...     ...
Yukon             OK            0.10    0.10
Zeigler           IL           -0.17    0.28
Zenda             KS           -0.28    0.17
Zionsville        IN            0.17    0.27
Zirconia          NC            0.38    0.46

[4965 rows x 2 columns]

3. Answer
  
  a. First we group by state and city (to account for cities that may have the same names in different states) and take the mean of the ZHVI. We sort so that the highest numbers are at the top.

In [105]:
zhvi_cities = z_merged.groupby(['StateName', 'City']).zhvi.mean().sort_values(ascending=False)

3. b. We do the same for median income.

In [103]:
hhi_cities = z_merged.groupby(['StateName', 'City']).median_hhi.mean().sort_values(ascending=False)

3. c. For each of these, we take the top 100 rows (after resetting the index).

In [113]:
top_zhvi_cities = zhvi_cities.reset_index().head(100)
top_hhi_cities = hhi_cities.reset_index().head(100)

3. d. Now we can use `merge` to find those cities in the top 100 for ZHVI that are also in the top 100 for median income. Since the `StateName` and `City` columns are common to both of the DataFrames we're mering we don't have to specify them as the keys for merging -- pandas will use them by default.

In [114]:
top_zhvi_cities.merge(top_hhi_cities)

,StateName,City,zhvi,median_hhi
0,CA,Atherton,"6,300,275.67","250,001.00"
1,CA,Portola Valley,"3,737,899.75","220,625.00"
2,CA,Los Altos,"3,422,485.42","226,748.00"
3,CA,Ross,"3,020,391.08","250,001.00"
4,WA,Medina,"2,874,184.92","192,120.00"
5,CA,Saratoga,"2,764,594.08","177,041.00"
6,FL,Fisher Island,"2,671,508.25","223,618.00"
7,CA,Los Gatos,"2,269,209.00","166,740.00"
8,NY,Old Westbury,"2,147,580.75","192,266.00"
9,CA,Orinda,"2,115,631.92","208,640.00"
